# Bert를 사용한 괄호 단어 찾기. 사용자 vocab을 사용한

2개의 채팅 문답에서 한 부분을 괄호 치고, 이 괄호 친 부분의 단어를 예측.

```
 하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 감정의 일부니까요.

 하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 (??)의 일부니까요.
```



copy from https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/master/7.PRETRAIN_METHOD/7.2.2.bert_finetune_KorNLI.ipynb

# 필요 라이브러리 설치

In [1]:
!pip install transformers==3.0.2
!pip install sentencepiece

     |████████████████████████████████| 769 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 1.2 MB 58.4 MB/s 
     |████████████████████████████████| 3.0 MB 58.8 MB/s 


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 5.5 MB/s 
     |████████████████████████████████| 448 kB 65.4 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# 셋업

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [4]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'
CUSTOM_VOCAB_FILE = 'custom_vocab.txt'
MAX_VOCAB_SIZE = 10000*2

# 데이터

## 데이터 다운로드

In [5]:
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv

--2021-11-21 02:00:19--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  --.-KB/s    in 0.03s   

2021-11-21 02:00:20 (28.3 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]



In [6]:
!head ChatbotData.csv

Q,A,label
12시 땡!,하루가 또 가네요.,0
1지망 학교 떨어졌어,위로해 드립니다.,0
3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
PPL 심하네,눈살이 찌푸려지죠.,0
SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0


## 데이터 로딩

In [7]:
df = pd.read_csv("ChatbotData.csv")

In [8]:
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## 데이터 섞기

In [9]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,Q,A,label
0,하루종일 썸남 생각만 해. 괜찮을까?,그것 또한 감정의 일부니까요.,2
1,냉장고에 먹을 게 하나도 없네,슈퍼라도 가서 쇼핑하고 오세요.,0
2,인사드리러 갔는데 파혼하는게 나을것 같아,이혼이 아니라 다행입니다.,0
3,금값 어때,비싸요.,0
4,연애상담하더니 둘이 사귀더라,대화를 하다가 친해졌나봐요.,2


## 필요 입출력 값 준비

In [10]:
sentences1 = df.Q.values.copy().astype(np.str)
sentences2 = df.A.values.copy().astype(np.str)
relation = df.label.values.copy().astype(np.int)

In [11]:
print(sentences1.shape)
print(sentences2.shape)
print(relation.shape)

(11823,)
(11823,)
(11823,)


필요 시, 실습 시간 관계로 전체 중에 일부 만 사용한다.

In [12]:
SHORT_COUNT = 10000*10
sentences1 = sentences1[:SHORT_COUNT]
sentences2 = sentences2[:SHORT_COUNT]

## Vocab 파일 만들기

In [13]:
all_sentence = []
all_sentence.extend(sentences1)
all_sentence.extend(sentences2)

In [14]:
print(len(all_sentence))
print(all_sentence[0])

23646
하루종일 썸남 생각만 해. 괜찮을까?


### vocab builder 생성

In [15]:
from konlpy.tag import Okt
import collections
from collections import OrderedDict

BERT_PREFIX = "##"

class KonlpyVocabMaker():

  def __init__(self, texts):
    self._tokens = []
    self._tokenize(texts)

  # texts = '하늘이 푸른가요? 나는 푸른색이 좋아요'
  # return ['하늘', '##이', '푸른가요', '?', '나', '##는', '푸른색', '##이', '좋아요']
  def _tokenize(self, texts):
    tokenizer = Okt()

    def _has_preceding_space(text, token, last_position):
      return text[last_position:].startswith(" "+token)

    def _tokenize_a_text(text):
      poses = tokenizer.pos(text)

      tokens = []

      last_position = 0
      for i, pos in enumerate(poses):
        org_token = pos[0]
        token = pos[0]
        if i==0:
          pass
        elif pos[1]=="Punctuation":
          if _has_preceding_space(text, token, last_position):  # " 'of"
            last_position += 1
          else:                                                 # "습니다."
            pass
        elif _has_preceding_space(text, token, last_position):
          last_position += 1
        elif pos[1]=="Alpha":
          pass
        else:
          token = BERT_PREFIX+token 

        tokens.append(token)
        last_position += len(org_token)

      return tokens

    # 각 문장별로 토크나이징
    all_tokens = []  
    for text in tqdm(texts):
      all_tokens.extend(_tokenize_a_text(text))

    # 빈도 순으로 정열
    counts = collections.Counter(all_tokens)
    sorted_tokens = sorted(all_tokens, key=counts.get, reverse=True)

    # 단어 중복 삭제
    sorted_tokens = list(OrderedDict.fromkeys(sorted_tokens))

    # Bert의 4개 특수 토큰을 삽입
    sorted_tokens.insert(0, '[PAD]')
    sorted_tokens.insert(1, '[UNK]')
    sorted_tokens.insert(2, '[CLS]')
    sorted_tokens.insert(3, '[SEP]')
    sorted_tokens.insert(4, '[MSK]')

    self._tokens = sorted_tokens

  def get_vocab(self):
    return self._tokens
  

In [16]:
konlply_tokenizer = KonlpyVocabMaker(['견인 회사는 "주권"으로 명명되었다.'])
vocab = konlply_tokenizer.get_vocab()
print(vocab)

konlply_tokenizer = KonlpyVocabMaker(['하늘이 푸른가요? 나는 푸른색이 좋아요'])
vocab = konlply_tokenizer.get_vocab()
print(vocab)

100%|██████████| 1/1 [00:06<00:00,  6.07s/it]


['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MSK]', '"', '견인', '회사', '##는', '##주권', '##으로', '##명명', '##되었다', '.']


100%|██████████| 1/1 [00:00<00:00, 26.82it/s]

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MSK]', '##이', '하늘', '푸른가요', '?', '나', '##는', '푸른색', '좋아요']


### 토크나이징 실행

In [17]:
konlply_tokenizer = KonlpyVocabMaker(all_sentence)

vocab = konlply_tokenizer.get_vocab()


100%|██████████| 23646/23646 [00:24<00:00, 947.82it/s]


In [18]:
print("org vocab size =",len(vocab))
vocab = vocab[:MAX_VOCAB_SIZE]
vocab_size = len(vocab)
print("vocab_size = ", len(vocab))

org vocab size = 13974
vocab_size =  13974


In [19]:
print(vocab[:20])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MSK]', '.', '##이', '##가', '##을', '?', '거', '사람', '##에', '##예요', '##은', '##도', '##요', '##를', '사랑', '생각']


### vocab 파일 저장

In [20]:
with open(CUSTOM_VOCAB_FILE, 'w') as f:
  for item in vocab:
    f.write("%s\n" % item)

In [21]:
!wc {CUSTOM_VOCAB_FILE}

 13974  13974 142422 custom_vocab.txt


## Tokenizer 생성

In [22]:
print(sentences1[0], sentences2[0])

하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 감정의 일부니까요.


In [23]:
tokenizer = BertTokenizer(vocab_file=CUSTOM_VOCAB_FILE, do_lower_case=False, model_max_length=SEQ_LENGTH)

In [24]:
tokenized = tokenizer(sentences1[0], text_pair=sentences2[0], max_length=30, padding='max_length')
print("original sentence  :", sentences1[0], sentences2[0])
print("tokens             :", tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print("token id           :", tokenized['input_ids'])
print("attention mask     :", tokenized['attention_mask'])
print("token type         :", tokenized['token_type_ids'])

original sentence  : 하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 감정의 일부니까요.
tokens             : ['[CLS]', '하루', '##종일', '썸남', '생각', '##만', '해', '.', '괜찮을까', '?', '[SEP]', '그것', '또한', '감정', '##의', '일부', '##니까', '##요', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
token id           : [2, 201, 2202, 255, 19, 42, 84, 5, 630, 9, 3, 1052, 3281, 174, 22, 2890, 387, 16, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention mask     : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
token type         : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


![bert_input_architecture](https://user-images.githubusercontent.com/1250095/50039788-8e4e8a00-007b-11e9-9747-8e29fbbea0b3.png)

## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [25]:
MASK_ID = vocab.index('[MSK]')
print(MASK_ID)

4


In [26]:
import logging
logging.basicConfig(level=logging.ERROR)

In [27]:
from random import randrange

MASK_ID = vocab.index('[MSK]')
MASK_TYPE_ID = 2

def is_special_token(token_id):
  return tokenizer.convert_ids_to_tokens(token_id).startswith("[")

def replace_a_token_as_mask(tokenized):

  input_ids = tokenized['input_ids']
  attention_mask = tokenized['attention_mask']
  token_type_ids = tokenized['token_type_ids']

  last_index = attention_mask.index(0) - 1
  mask_index = None
  while mask_index==None:
    i = randrange(last_index)
    if not is_special_token(input_ids[i]):
      mask_index = i
  replaced_value = input_ids[mask_index]
  input_ids[mask_index] = MASK_ID
  token_type_ids[mask_index] = MASK_TYPE_ID

  return tokenized, replaced_value


def build_model_input_output(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []
  labels = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length', )
    tokenized, label = replace_a_token_as_mask(tokenized)
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])
    labels.append(label)

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids)), np.array(labels)

In [28]:
MAX_DATA_COUNT = 1000*100
x, y = build_model_input_output(sentences1[:MAX_DATA_COUNT], sentences2[:MAX_DATA_COUNT])

In [29]:
input_ids = x[0][0]
print("original sentence  :", sentences1[0], sentences2[0])
print("masked tokens      :", tokenizer.convert_ids_to_tokens(input_ids)[:20])

original sentence  : 하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 감정의 일부니까요.
masked tokens      : ['[CLS]', '하루', '##종일', '썸남', '생각', '##만', '[MSK]', '.', '괜찮을까', '?', '[SEP]', '그것', '또한', '감정', '##의', '일부', '##니까', '##요', '.', '[SEP]']


x는 다음과 같이 구성됨
```
x = (  token_ids,  attention_masks,  token_types   )
       x[0],       x[1],             [2]
```

<br>

첫번 째 데이터는 
```
   ( token_ids[0],  attention_masks[0], token_types[0] )
 = ( x[0][0],       x[1][0],            x[2][0]  )
```


## train/test 분리

In [30]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

In [31]:
print(sentences1[0], sentences2[0])
print(tokenizer.decode(train_x[0][0][:35]))
print(train_x[0][0][:35])
print(train_x[1][0][:35])
print(train_x[2][0][:35])

하루종일 썸남 생각만 해. 괜찮을까? 그것 또한 감정의 일부니까요.
[CLS] 하루종일 썸남 생각만 [MSK]. 괜찮을까? [SEP] 그것 또한 감정의 일부니까요. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[   2  201 2202  255   19   42    4    5  630    9    3 1052 3281  174
   22 2890  387   16    5    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 2 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# 학습

## 모델 생성

In [44]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertClassifier(tf.keras.Model):
  def __init__(self):
    super(TFBertClassifier, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME, trainable=True)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.classifier = Dense(vocab_size, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range), 
                            name="classifier", activation="softmax")

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.classifier(v)

    return out

model = TFBertClassifier()


참고로 Bert의 default 설정은 다음과 같다.

In [45]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [46]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


## 학습 실행

In [47]:
print(train_y.shape)

(9458,)


In [48]:
history = model.fit(train_x, train_y, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
266/266 [==============================] - 145s 503ms/step - loss: 8.0608 - accuracy: 0.1097 - val_loss: 7.2592 - val_accuracy: 0.1152
Epoch 2/10
266/266 [==============================] - 129s 487ms/step - loss: 6.7596 - accuracy: 0.1143 - val_loss: 7.2104 - val_accuracy: 0.1152
Epoch 3/10
266/266 [==============================] - 130s 487ms/step - loss: 6.5498 - accuracy: 0.1171 - val_loss: 7.2167 - val_accuracy: 0.1300
Epoch 4/10
266/266 [==============================] - 129s 487ms/step - loss: 6.4192 - accuracy: 0.1229 - val_loss: 7.1855 - val_accuracy: 0.1332
Epoch 5/10
266/266 [==============================] - 129s 486ms/step - loss: 6.2942 - accuracy: 0.1340 - val_loss: 7.2280 - val_accuracy: 0.1342
Epoch 6/10
266/266 [==============================] - 129s 487ms/step - loss: 6.1865 - accuracy: 0.1414 - val_loss: 7.1968 - val_accuracy: 0.1406
Epoch 7/10
266/266 [==============================] - 129s 487ms/step - loss: 6.0773 - accuracy: 0.1446 - val_loss: 7.1805 -

In [49]:
loss, acc = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("acc =", acc)

74/74 [==============================] - 13s 169ms/step - loss: 7.2420 - accuracy: 0.1349
loss = 7.24198579788208
acc = 0.13488371670246124


## 분류 실행

In [59]:
def do_classify(sentence1, sentence2):
  model_input, label = build_model_input_output([sentence1], [sentence2])
  input = tokenizer.decode(model_input[0][0])
  input = input.replace(" [PAD]", "")
  input = input.replace("[CLS] ", "")
  input = input.replace(" [SEP]", "")
  input = input.replace(" [MSK]", "(???)")
  y_ = model.predict(model_input)
  predicted = y_[0].argsort()[-5:][::-1]
  print(input, "-->",  "expected :", [vocab[i] for i in predicted])

do_classify("연애상담하더니 둘이 사귀더라", "대화를 하다가 친해졌나봐요.")
do_classify("연애상담하더니 둘이 사귀더라", "대화를 하다가 친해졌나봐요.")
do_classify("연애상담하더니 둘이 사귀더라", "대화를 하다가 친해졌나봐요.")
do_classify("연애상담하더니 둘이 사귀더라", "대화를 하다가 친해졌나봐요.")

연애상담(???) 둘이 사귀더라 대화를 하다가 친해졌나봐요. --> expected : ['##가', '##이', '안', '##에', '##의']
연애상담하더니 둘이 사귀더라 대화를 하다가 친해졌나봐요(???) --> expected : ['.', '?', '!', '네', '##을']
연애상담하더니 둘이 사귀더라 대화를 하다가 친해졌나(???). --> expected : ['##예요', '##해보세요', '##요', '##봐요', '##보세요']
연애상담하더니 둘이 사귀더라 대화를(???) 친해졌나봐요. --> expected : ['있을', '생각', '될', '말', '해']


In [53]:
do_classify("SNS 맞팔 왜 안하지ㅠㅠ", "잘 모르고 있을 수도 있어요.")
do_classify("SNS 맞팔 왜 안하지ㅠㅠ", "잘 모르고 있을 수도 있어요.")
do_classify("SNS 맞팔 왜 안하지ㅠㅠ", "잘 모르고 있을 수도 있어요.")
do_classify("SNS 맞팔 왜 안하지ㅠㅠ", "잘 모르고 있을 수도 있어요.")
do_classify("SNS 맞팔 왜 안하지ㅠㅠ", "잘 모르고 있을 수도 있어요.")

SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도(???). --> expected : ['##예요', '##요', '##해보세요', '##보세요', '##봐요']
SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요(???) --> expected : ['.', '?', '!', '##을', '마음']
SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고(???) 수도 있어요. --> expected : ['거', '있을', '것', '생각', '될']
SNS(???) 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요. --> expected : ['##가', '##이', '##에', '안', '사람']
SNS(???) 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요. --> expected : ['##가', '##이', '##에', '안', '사람']
